In [40]:
import pandas as pd 
import math 

# Open excel file 
fixes = pd.read_excel('ScriptTest1_Fixes.xlsx')
raw_data = pd.read_csv('test1.csv')
raw_data = raw_data.rename(columns={ raw_data.columns[0]: "id" })

# Splice for useful information
fixes = fixes.loc[:,['Initial ID','Changed ID', 'Time First Seen']].dropna()

# Sort by Time 
fixes = fixes.sort_values(by=['Time First Seen'])

# Define functions 
# Finds and returns a car's initial ID given changed ID 
def FindRootId(id, graph):
    if(id not in graph):
        return np.nan
    current_id = id
    while(len(graph[current_id]) > 0): 
        current_id = graph[current_id][0]
    return current_id

# Is id1 connected to id2? 
def IsConnected(id1, id2, graph): 
    if(id1 not in graph or id2 not in graph): 
        return False 
    
    visited = [id1] 
    queue = [id1]
    while queue:          # Creating loop to visit each node
        m = queue.pop(0) 
        if(m == id2): 
            return True
        for neighbour in graph[m]:
          if(neighbour not in visited):
            visited.append(neighbour)
            queue.append(neighbour)
    return False 

# A fix is not valid if it implies id's were swapped  
def IsValidFix(initial_id, changed_id, graph): 
    if(initial_id == changed_id): 
        return True 
    # Check if the graph forms a cycle afer adding new edge from changed_id to initial_id
    if(changed_id in graph): 
        return not IsConnected(initial_id, changed_id, graph) 
    return True 

# Finds all rows to be changed and replaces id with initial_id 
def FixId(initial_id: int, changed_id: int, time_first_seen_seconds: float):
    margin_of_error = 1/60 
    minimum_time = time_first_seen_seconds - margin_of_error
    raw_data.loc[(raw_data['id'] == changed_id) & ((raw_data['time'] >= minimum_time)), 'id'] = initial_id

def PrintInvalidFixes(invalid_fixes): 
    print('WARNING - THE FOLLOWING FIXES WERE REDUDANT:')
    print('Index   Initial Id   Changed Id      Time First Seen')
    for index, initial_id, changed_id, time_first_seen in invalid_fixes:
        print(f'{index: 4d}: {initial_id:12d} {changed_id:12d} {time_first_seen: 20f}')
    print('\n')

def PrintRedundantFixes(redundant_fixes): 
    print('WARNING - THE FOLLOWING FIXES WERE REDUDANT: ')
    print('Index   Initial Id   Changed Id      Time First Seen')
    for index, initial_id, changed_id, time_first_seen in redundant_fixes:
        print(f'{index: 4d}: {initial_id:12d} {changed_id:12d} {time_first_seen: 20f}')
        
#Construct graph 
graph = {}
invalid_fixes = []
redundant_fixes = []

for index, row in fixes.iterrows():
    changed_id = int(row['Changed ID'])
    initial_id = int(row['Initial ID'])
    time_first_seen = float(row['Time First Seen'])
    row_data = (index + 1, initial_id, changed_id, time_first_seen)
    print(graph)
    if not IsValidFix(initial_id, changed_id, graph): 
        print("invalid")
        invalid_fixes.append(row_data)
        continue
        
    initial_id_actual = initial_id if initial_id not in graph else FindRootId(initial_id, graph)  
    changed_id_actual = changed_id if changed_id not in graph else FindRootId(changed_id, graph)
    
    if(initial_id_actual == changed_id_actual):
        redundant_fixes.append(row_data)
    else: 
        if(initial_id_actual not in graph): 
            graph[initial_id_actual] = []
        if(changed_id_actual not in graph): 
            graph[changed_id_actual] = []
        FixId(initial_id_actual, changed_id_actual, float(time_first_seen))
        graph[changed_id_actual] = [initial_id_actual]

PrintInvalidFixes(invalid_fixes) 
PrintRedundantFixes(redundant_fixes)

# raw_data.to_csv('output_01.csv', index=False)


{}
{2: [], 1: [2]}
{2: [], 1: [2], 41: [], 4: [41]}
{2: [], 1: [2], 41: [411], 4: [41], 411: []}
invalid
{2: [], 1: [2], 41: [411], 4: [41], 411: []}
Following fixes were not valid: 
Index   Initial Id   Changed Id      Time First Seen
   4:           41          411             1.033333


Index   Initial Id   Changed Id      Time First Seen
   5:          411            4             1.066667


In [ ]:
print('hello')